In [ ]:
import sys
from pathlib import Path

# Ajouter le root du projet au path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Importer sklearn metrics
from sklearn.metrics import confusion_matrix

# Importer les fonctions XGBoost et utilitaires
from ift6758.models.model_xgboost import (
    XGBoostModelTrainer, 
    generate_evaluation_plots,
    load_and_prepare_data
)
from ift6758.models.model_utils import (
    evaluate_model,
    print_metrics,
    plot_feature_importance,
    compare_models,
    plot_models_comparison
)
import adaboost as adb

# Configuration matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

In [2]:
# Configuration des chemins
MODELS_DIR = PROJECT_ROOT / "models saved"
FIGURES_DIR = PROJECT_ROOT / "figures" / "milestone2"

MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Charger et préparer les données automatiquement
X_train, y_train, X_val, y_val, X_test, y_test = load_and_prepare_data()

Donnees chargees!
  Train: 77,226 tirs | 18 features
  Val:   19,307 tirs | Target: 9.49% buts
  Test:  57,734 tirs


In [3]:
# Analyser le desequilibre des classes
from sklearn.utils.class_weight import compute_class_weight

# Distribution des classes
train_goal_rate = y_train.mean()
val_goal_rate = y_val.mean()
test_goal_rate = y_test.mean()

print("="*80)
print("ANALYSE DU DESEQUILIBRE DES CLASSES")
print("="*80)

print(f"\nTaux de buts (classe positive = 1):")
print(f"  - Train:      {train_goal_rate:.2%} ({y_train.sum():,} buts / {len(y_train):,} tirs)")
print(f"  - Validation: {val_goal_rate:.2%} ({y_val.sum():,} buts / {len(y_val):,} tirs)")
print(f"  - Test:       {test_goal_rate:.2%} ({y_test.sum():,} buts / {len(y_test):,} tirs)")

# Calculer le ratio de desequilibre
imbalance_ratio = (1 - train_goal_rate) / train_goal_rate
print(f"\nRatio de desequilibre: 1:{imbalance_ratio:.1f}")
print(f"   Il y a {imbalance_ratio:.1f}x plus de non-buts que de buts")

# Calculer scale_pos_weight pour XGBoost
scale_pos_weight = imbalance_ratio
print(f"\nRecommandation pour XGBoost:")
print(f"   scale_pos_weight = {scale_pos_weight:.2f}")
print(f"   (donne plus de poids aux exemples de buts)")

ANALYSE DU DESEQUILIBRE DES CLASSES

Taux de buts (classe positive = 1):
  - Train:      9.49% (7,325 buts / 77,226 tirs)
  - Validation: 9.48% (1,831 buts / 19,307 tirs)
  - Test:       9.76% (5,636 buts / 57,734 tirs)

Ratio de desequilibre: 1:9.5
   Il y a 9.5x plus de non-buts que de buts

Recommandation pour XGBoost:
   scale_pos_weight = 9.54
   (donne plus de poids aux exemples de buts)
